# Setting and Load library

## import library

In [49]:
############################################################################
# python default library
############################################################################
import os
import glob
import sys
import random
############################################################################
############################################################################
# additional library
############################################################################
# general analysis tool-kit
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# sound analysis tool-kit
import librosa
import librosa.core
import librosa.feature

# pytorch
import torch
import torch.utils.data as data
from torch import optim, nn
from torch.utils.data.dataset import Subset

# pytorch ignite
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.contrib.handlers.tensorboard_logger import *

# deeplearning tool-kit
from torchvision import transforms

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
############################################################################
# original library
############################################################################
import common as com
from pytorch_model import AutoEncoder

## setting seed

In [50]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [51]:
set_seed(42)

## setting I/O path 

In [52]:
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [53]:
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
eval_test_path = INPUT_ROOT + "/eval_test"
# machine type
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']

In [22]:
dev_train_paths = {}
dev_test_paths = {}
add_train_paths = {}
add_test_paths = {}
eval_test_paths = {}

for machine_type in enumerate(machine_types):
    
    dev_train_all_paths = ["{}/{}/train/".format(dev_path, machine_type) + 
                           file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths[machine_type] = {}
    dev_train_paths[machine_type]['train'], 
    dev_train_paths[machine_type]['valid'] = train_test_split(dev_train_all_paths,
                                                              test_size=config['eval_param']['test_size'],
                                                              random_state=config['eval_param']['random_state'])
    #dev_test_paths[machine_type] = ["{}/{}/test/".format(dev_path, machine_type) + 
    #                                 file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    add_train_paths[machine_type] = ["{}/{}/train/".format(add_dev_path, machine_type) + 
                                     file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    #add_test_paths[machine_type] = ["{}/{}/test/".format(add_dev_path, machine_type) + 
    #                                 file for file in os.listdir("{}/{}/test".format(add_dev_path, machine_type))]
    #eval_test_paths[machine_type] = ["{}/{}/test/".format(eval_test_path, machine_type) + 
    #                                 file for file in os.listdir("{}/{}/test".format(eval_test_path, machine_type))]

In [ ]:
for machine_type in machine_types:
    

In [33]:
len(dev_train_paths[machine_type])

3291

# Data Load and Preprocessing

In [23]:
class Wav_to_Melspectrogram(object):
    """
    wavデータロード(波形) -> ログメルスペクトログラム
    
    Attributes
    ----------
    dims = n_mels * frames
    sound_data : numpy.ndarray.shape = (timecourse, dims)
    """
    def __init__(self, sound_data=None):
        self.sound_data = sound_data
    
    def __call__(self, sample):
        self.sound_data = com.file_to_vector_array(
            sample['wav_name'],
            config['mel_spectrogram_param']['n_mels'],
            config['mel_spectrogram_param']['frames'],
            config['mel_spectrogram_param']['n_fft'],
            config['mel_spectrogram_param']['hop_length'],
            config['mel_spectrogram_param']['power']
        )
        self.labels = np.full((self.sound_data.shape[0]), sample['label'])
        return {'features': self.sound_data, 'labels': self.labels}

In [24]:
class ToTensor(object):
    """
    Convert ndarrays in sample to Tensors.
    """

    def __call__(self, sample):
        features, labels = sample['features'], sample['labels']
        
        return {'features': torch.from_numpy(features), 'labels': torch.from_numpy(labels)}

In [25]:
class DCASE_task2_Dataset(torch.utils.data.Dataset):
    '''
    Attribute
    ----------
    
    '''
    
    def __init__(self, file_list, transform=None):
        self.transform = transform
        self.file_list = file_list
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        # ファイル名でlabelを判断
        if "normal" in file_path:
            label = 0
        else:
            label = 1
        
        sample = {'wav_name':file_path, 'label':label}
        sample = self.transform(sample)
        
        return sample

In [26]:
transform = transforms.Compose([
    Wav_to_Melspectrogram(),
    ToTensor()
])
train_dataset = DCASE_task2_Dataset(dev_train_paths[machine_types[0]], transform=transform)
#valid_dataset = DCASE_task2_Dataset(dev_valid_paths[machine_types[0]], transform=transform)

In [27]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=config['fit']['batch_size'],
    shuffle=config['fit']['shuffle'],
    )

valid_loader = torch.utils.data.DataLoader(
    dataset=valid_dataset,
    batch_size=config['fit']['batch_size'],
    shuffle=False,
    )

In [31]:
train_dataset[0]['features'].shape

torch.Size([309, 640])

# train

In [29]:
net = AutoEncoder()
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

In [ ]:
def train_net(net, train_loader, valid_loader, criterion, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
    print("use:", device)
    net.to(device)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('----------------------')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
            
            epoch_loss = 0.0